## Import

In [2]:
%matplotlib auto

from PIL import Image
from urllib.request import urlopen
from io import BytesIO

import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

import cv2

Using matplotlib backend: <object object at 0x7f0741ccbd00>


In [4]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

### Sample code to open and resize a photo

In [ ]:
im = cv2.imread('photo.jpg')

im_res = ResizeWithAspectRatio(im, width=1280)
cv2.imshow('frame', im_res)
cv2.waitKey(8000)
cv2.destroyAllWindows()

## Perspective Transform with L2 norm

Insert path to photo and 4 points below

In [ ]:
# path to photo
im = cv2.imread('photos/20230915_162042.jpg')
gray_img=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
thresh_img=cv2.adaptiveThreshold(gray_img, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 3)

#cnts = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# 4 edge points
pt_A = [760, 1943]
pt_B = [3990, 1926]
pt_C = [774, 2534]
pt_D = [3981, 2514]

# Here, I have used L2 norm. You can use L1 also.
width_AB = np.sqrt(((pt_A[0] - pt_B[0]) ** 2) + ((pt_A[1] - pt_B[1]) ** 2))
width_CD = np.sqrt(((pt_C[0] - pt_D[0]) ** 2) + ((pt_C[1] - pt_D[1]) ** 2))
maxWidth = max(int(width_AB), int(width_CD))
 
 
height_AC = np.sqrt(((pt_A[0] - pt_C[0]) ** 2) + ((pt_A[1] - pt_C[1]) ** 2))
height_BD = np.sqrt(((pt_B[0] - pt_D[0]) ** 2) + ((pt_B[1] - pt_D[1]) ** 2))
maxHeight = max(int(height_AC), int(height_BD))

print(maxWidth)
print(maxHeight)

pts1 = np.float32([pt_A, pt_B,
                    pt_C, pt_D])
pts2 = np.float32([[0, 0], [maxWidth, 0],
                [0, maxHeight], [maxWidth, maxHeight]])
    
# Apply Perspective Transform Algorithm
matrix = cv2.getPerspectiveTransform(pts1, pts2)
result = cv2.warpPerspective(im, matrix,(maxWidth, maxHeight),flags=cv2.INTER_LINEAR)
    
# Wrap the transformed image
im_res = ResizeWithAspectRatio(im, width=1280)
res_res = ResizeWithAspectRatio(result, width=1280)
thresh_img_res = ResizeWithAspectRatio(thresh_img, width=1280)
cv2.imshow('frame2', res_res)

# Wait time before close
cv2.waitKey(8000)
cv2.destroyAllWindows()

### Write a transformed temprorary png 

In [5]:
cv2.imwrite('temp.png',result)

True

### Threshold

In [ ]:
img_rgb = cv2.imread('temp.png')

assert img_rgb is not None, "file could not be read, check with os.path.exists()"
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('templates/2_rec_m.png', cv2.IMREAD_GRAYSCALE)
assert template is not None, "file could not be read, check with os.path.exists()"
w, h = template.shape[::-1]


print(w)
print(h)

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.65
loc = np.where( res >= threshold)

for pt in zip(*loc[::-1]):
 img_rgb[pt[1]+(h//2), pt[0]+(w//2)] = [0,0,255]

img_rgb = ResizeWithAspectRatio(img_rgb, width=1280)
img_gray = ResizeWithAspectRatio(img_gray, width=1280)
res = ResizeWithAspectRatio(res, width=1280)
cv2.imshow('image', res)
cv2.imshow('image1', img_rgb)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [ ]:
img_rgb = cv2.imread('res.png')

assert img_rgb is not None, "file could not be read, check with os.path.exists()"
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('templates/2_rec_m.png', cv2.IMREAD_GRAYSCALE)
assert template is not None, "file could not be read, check with os.path.exists()"
w, h = template.shape[::-1]

print(w)
print(h)

sh_h, sh_w, sh_c = img_rgb.shape
print(sh_h, sh_w)

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
res_h, res_w = res.shape[:2]

threshold = 0.7#0.6975
max_val = 1
centers = []
while max_val > threshold:
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    if max_val > threshold:
        centers.append( (max_loc[0] + w//2, max_loc[1] + h//2) )

        x1 = max(max_loc[0] - w//2, 0)
        y1 = max(max_loc[1] - h//2, 0)

        x2 = min(max_loc[0] + w//2, res_w)
        y2 = min(max_loc[1] + h//2, res_h)

        res[y1:y2, x1:x2] = 0

        img_rgb = cv2.rectangle(img_rgb,(max_loc[0],max_loc[1]), (max_loc[0]+w+1, max_loc[1]+h+1), (0,0,255), 3)

# coordinates of rectangle centers
print(centers)
print(img_rgb.shape)

img_rgb = ResizeWithAspectRatio(img_rgb, width=1280)
img_gray = ResizeWithAspectRatio(img_gray, width=1280)
res = ResizeWithAspectRatio(res, width=1280)
cv2.imshow('image', res)
cv2.imshow('image1', img_rgb)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [31]:
centers.append((1340, 397))
#centers.append((3309, 735))

In [7]:
img_rgb.shape

(585, 1280, 3)

## Transform data to new scale

Enter [min, max] of x and y axes

In [ ]:
x = [0, 600]
y = [0, 1]

In [36]:
df = pd.DataFrame([((i[0]/sh_w)*x[1], y[1]-(i[1]/sh_h*(y[1]-y[0]))) for i in centers])

In [37]:
df.columns = ["x","y"]

Enter name of a csv

In [38]:
df.to_csv("20230915_162042.csv")

Check graph

In [ ]:
sns.scatterplot(data=df, x="x", y="y")